In [ ]:
#https://github.com/TsingZ0/PFL-Non-IID

In [1]:
import pandas as pd
from collections import OrderedDict
from typing import OrderedDict
import numpy as np
import copy
import os
import torch
import torch.nn as nn
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torch.utils.data import DataLoader, Subset, random_split

from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, Subset, random_split
import torch.utils.data as data_utils
import pickle

In [2]:
# detail about the state: load_acs.py

In [3]:
import os
import shutil

client_df_path="D:/Download/pythonProject/HiWi/pFedLA_Folktable/clients_df/"

    
for file in os.listdir(client_df_path):
    file_path = os.path.join(client_df_path, file)
    os.remove(file_path)
    print(f"Deleted: {file_path}")
        


print("All files removed successfully.")

All files removed successfully.


In [4]:
from folktables import ACSDataSource, ACSEmployment,ACSIncome
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["AL"], download=True)

In [5]:
# features_np, label_np, group_np = ACSEmployment.df_to_numpy(acs_data)
features, label, group = ACSEmployment.df_to_pandas(acs_data)

In [6]:
features

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0


In [7]:
label

,ESR
0,False
1,False
2,False
3,False
4,False
...,...
47772,True
47773,False
47774,False
47775,False


In [8]:
merged_df=pd.concat([features, label], axis=1)
merged_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False


In [9]:
len(merged_df)

47777

In [10]:
group.value_counts()

RAC1P
1        35551
2        10115
9          835
6          556
8          491
3          182
5           33
7           13
4            1
dtype: int64

In [11]:
merged_df["RAC1P"].value_counts()

# 1: White alone – 2: Black or African American alone

1.0    35551
2.0    10115
9.0      835
6.0      556
8.0      491
3.0      182
5.0       33
7.0       13
4.0        1
Name: RAC1P, dtype: int64

In [12]:
merged_df["SEX"].value_counts()

2.0    24805
1.0    22972
Name: SEX, dtype: int64

In [13]:
merged_df

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,19.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
1,18.0,18.0,5.0,17.0,2.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
2,53.0,17.0,5.0,16.0,1.0,0.0,1.0,1.0,4.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,False
3,28.0,19.0,5.0,16.0,2.0,0.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
4,25.0,12.0,5.0,16.0,1.0,0.0,1.0,3.0,4.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47772,18.0,16.0,5.0,2.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
47773,15.0,11.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47774,10.0,6.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,False
47775,4.0,1.0,5.0,2.0,2.0,7.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,0.0,1.0,1.0,False


In [14]:
shuffled = merged_df.sample(frac=1)
shuffled

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
30788,11.0,8.0,5.0,2.0,2.0,1.0,1.0,1.0,0.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,False
19274,60.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,True
24198,21.0,16.0,5.0,4.0,2.0,0.0,1.0,3.0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,True
28683,35.0,21.0,1.0,0.0,2.0,0.0,1.0,3.0,4.0,4.0,1.0,2.0,2.0,2.0,1.0,1.0,True
17241,24.0,21.0,5.0,0.0,2.0,0.0,5.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,6.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,32.0,20.0,5.0,0.0,2.0,0.0,1.0,1.0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,True
32566,74.0,21.0,1.0,5.0,2.0,0.0,4.0,1.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,False
39460,71.0,13.0,2.0,0.0,2.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,False
25884,15.0,11.0,5.0,2.0,2.0,1.0,1.0,1.0,0.0,4.0,1.0,2.0,2.0,2.0,1.0,2.0,False


In [15]:
import numpy as np

total_number = len(merged_df)
print("total_number: ",total_number)

num_parts = 10
lower_limit = 3000
upper_limit = 6000

# Generate 9 random numbers between lower_limit and upper_limit
random_numbers = np.random.randint(lower_limit, upper_limit + 1, num_parts - 1)

# Sort the random numbers to ensure variety and add the remaining to the last number
random_numbers = np.sort(random_numbers)
random_numbers = np.append(random_numbers, total_number - np.sum(random_numbers))

print("Random numbers:", random_numbers)
print("Sum:", np.sum(random_numbers))


total_number:  47777
Random numbers: [3134 3341 4014 4127 4790 4882 5191 5198 5318 7782]
Sum: 47777


In [16]:
all_df_split=np.split(merged_df.sample(frac=1, random_state=42), np.cumsum(random_numbers)[:-1])

In [17]:
temp=0
for i in all_df_split:
    print("len of Dfs",len(i))
    temp+=len(i)
    
temp

len of Dfs 3134
len of Dfs 3341
len of Dfs 4014
len of Dfs 4127
len of Dfs 4790
len of Dfs 4882
len of Dfs 5191
len of Dfs 5198
len of Dfs 5318
len of Dfs 7782


47777

In [19]:
list_client_df=[]

output_directory="D:/Download/pythonProject/HiWi/pFedLA_Folktable/clients_df/"

for idx, ind_df in enumerate(all_df_split):
    encoder = LabelEncoder()
    x_train=0
    y_train=0
    
    x_train = ind_df[['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC','NATIVITY', 'DEAR', 'DEYE', 'DREM','SEX', 'RAC1P']]
    y_train = ind_df[['ESR']]
    
    output_filename = os.path.join(output_directory, f"client_{idx}.csv")
    ind_df.to_csv(output_filename, index=False)
    
    x_train = preprocessing.normalize(x_train)
    encoder.fit(y_train)
    y_train = encoder.transform(y_train)
    
    X_train= torch.tensor(x_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    
    final_data = data_utils.TensorDataset(X_train, y_train)
    
    list_client_df.append(final_data)

C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\p

In [20]:
len(all_df_split)

10

In [38]:
valset_ratio=0.05
testset_ratio=0.25
BATCH=64

client_data_for_training=[]
client_data_for_validation=[]
client_data_for_testing=[]

for i in range(len(all_df_split)):
    val_samples_num = int(len(list_client_df[i]) * valset_ratio)
    test_samples_num = int(len(list_client_df[i]) * testset_ratio)
    train_samples_num = len(list_client_df[i]) - val_samples_num - test_samples_num


    trainset, valset, testset = random_split(
            list_client_df[i], [train_samples_num, val_samples_num, test_samples_num])

    trainloader = data_utils.DataLoader(trainset, batch_size=BATCH, shuffle=True)
    valloader = data_utils.DataLoader(valset, batch_size=BATCH)
    testloader = data_utils.DataLoader(testset, batch_size=BATCH)

    client_data_for_training.append(trainloader)
    client_data_for_validation.append(valloader)
    client_data_for_testing.append(testloader)

    

    print(i,"::",len(list_client_df[i]),"::",val_samples_num," :: ",test_samples_num," :: ",train_samples_num)

0 :: 3134 :: 156  ::  783  ::  2195
1 :: 3341 :: 167  ::  835  ::  2339
2 :: 4014 :: 200  ::  1003  ::  2811
3 :: 4127 :: 206  ::  1031  ::  2890
4 :: 4790 :: 239  ::  1197  ::  3354
5 :: 4882 :: 244  ::  1220  ::  3418
6 :: 5191 :: 259  ::  1297  ::  3635
7 :: 5198 :: 259  ::  1299  ::  3640
8 :: 5318 :: 265  ::  1329  ::  3724
9 :: 7782 :: 389  ::  1945  ::  5448


In [22]:
len(list_client_df[0])

3134

In [23]:
with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/clients_training.pkl", "wb") as f:
    pickle.dump(client_data_for_training, f)
    
with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/clients_validation.pkl", "wb") as f:
    pickle.dump(client_data_for_validation, f)
    
with open("D:/Download/pythonProject/HiWi/pFedLA_Folktable/clients_testing.pkl", "wb") as f:
    pickle.dump(client_data_for_testing, f)

In [24]:
len(client_data_for_testing),len(client_data_for_training),len(client_data_for_validation)

(10, 10, 10)

In [29]:
len(trainset[0])

35

In [33]:
client_9="D:/Download/pythonProject/HiWi/pFedLA_Folktable/clients_df/client_9.csv"
df9=pd.read_csv(client_9)
df9

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,52.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,False
1,59.0,16.0,1.0,1.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
2,5.0,2.0,5.0,2.0,2.0,1.0,1.0,1.0,0.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,False
3,80.0,12.0,2.0,0.0,1.0,0.0,1.0,1.0,4.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,False
4,67.0,20.0,1.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7777,38.0,21.0,1.0,0.0,2.0,0.0,1.0,3.0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,False
7778,9.0,6.0,5.0,2.0,1.0,5.0,1.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,False
7779,76.0,16.0,3.0,0.0,2.0,0.0,1.0,1.0,4.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,False
7780,18.0,18.0,5.0,17.0,1.0,0.0,1.0,3.0,4.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,False


In [35]:
df9["ESR"].value_counts()


False    4622
True     3160
Name: ESR, dtype: int64

In [40]:
with open("..\\clients_training.pkl", "rb") as f:
       trainset = pickle.load(f)

In [42]:
# Assuming you have a DataLoader named 'dataloader'
for batch_data, batch_labels in trainset[9]:
    print("Batch Data:", batch_data)
    print("Batch Labels:", batch_labels)


Batch Data: tensor([[0.9222, 0.3721, 0.0324,  ..., 0.0324, 0.0162, 0.0162],
        [0.9261, 0.3519, 0.0741,  ..., 0.0185, 0.0370, 0.0185],
        [0.9433, 0.3144, 0.0143,  ..., 0.0286, 0.0143, 0.0143],
        ...,
        [0.9207, 0.3593, 0.0225,  ..., 0.0449, 0.0449, 0.0449],
        [0.9716, 0.2130, 0.0266,  ..., 0.0266, 0.0266, 0.0266],
        [0.1325, 0.0000, 0.6623,  ..., 0.0000, 0.1325, 0.1325]])
Batch Labels: tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
  

Batch Data: tensor([[0.6532, 0.3266, 0.4082,  ..., 0.1633, 0.0816, 0.0816],
        [0.9337, 0.3381, 0.0483,  ..., 0.0161, 0.0161, 0.0161],
        [0.9702, 0.2186, 0.0137,  ..., 0.0273, 0.0273, 0.0137],
        ...,
        [0.6544, 0.6544, 0.1722,  ..., 0.0689, 0.0689, 0.0689],
        [0.7145, 0.4547, 0.3248,  ..., 0.1299, 0.0650, 0.0650],
        [0.9703, 0.2334, 0.0123,  ..., 0.0246, 0.0123, 0.0123]])
Batch Labels: tensor([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
  